In [1]:
import numpy as np

import pandas as pd

import kagglehub

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.mixture import GaussianMixture

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.metrics import silhouette_score, davies_bouldin_score

In [2]:

# Define the dataset path
dataset_path = "/Users/Dataghost/.cache/kagglehub/datasets/vjchoudhary7/customer-segmentation-tutorial-in-python/versions/1/Mall_Customers.csv"

# Load the dataset
df = pd.read_csv(dataset_path, encoding="ISO-8859-1")

# Display the first few rows
print(df.head())


   CustomerID  Gender  Age  Annual Income (k$)  Spending Score (1-100)
0           1    Male   19                  15                      39
1           2    Male   21                  15                      81
2           3  Female   20                  16                       6
3           4  Female   23                  16                      77
4           5  Female   31                  17                      40


Step 2: Preprocessing

Remove unnecessary columns (CustomerID, Gender isn't needed for clustering).
Scale the features (StandardScaler ensures all values are on the same scale).

In [ ]:
# Select relevant features

features = df[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']]

In [ ]:
# Standardize the data

scaler = StandardScaler()

X_scaled = scaler.fit_transform(features)

Step 2: Determine the Optimal Number of Clusters

use Elbow Method (for K-Means) and AIC/BIC (for GMM).

Finding Optimal Clusters for K-Means (Elbow Method)


In [ ]:
wcss = []  # Within-cluster sum of squares

sil_scores =[]

for i in range(2,11):

    kmeans = KMeans(n_clusters=i, random_state=42, n_init=10)

    kmeans.fit(X_scaled)

    wcss.append(kmeans.inertia_)

    

